In [2]:
! pip install pytesseract pillow pandas pdf2image


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8
Err:1 http://security.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8
  404  Not Found [IP: 91.189.91.81 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/p/poppler/poppler-utils_22.02.0-2ubuntu0.8_amd64.deb  404  Not Found [IP: 91.189.91.81 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [3]:
!apt-get update
!apt-get install -y poppler-utils

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]           
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,111 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,209 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,271 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [48.5 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-dr

In [9]:
pip install pymupdf pdf2image pytesseract pillow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 72.5 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pytesseract
from pdf2image import convert_from_path

# Input / Output
pdf_path = "/kaggle/input/scanned-data-set/Revoult Bank statement.pdf"   # <-- your PDF
output_html = "bank_statement_full.html"

# Convert PDF → Images
pages = convert_from_path(pdf_path, dpi=200)
html_parts = []

for page_num, page in enumerate(pages, start=1):
    # OCR with line-level data
    data = pytesseract.image_to_data(page, output_type=pytesseract.Output.DICT)
    page_w, page_h = page.size

    # Page container
    html_parts.append(
        f"<div class='page' style='position:relative;width:{page_w}px;height:{page_h}px;"
        "border:1px solid #ccc;margin:20px auto;background:white;'>"
    )

    current_line = None
    line_text, line_left, line_top = [], 0, 0

    for i, word in enumerate(data["text"]):
        word = word.strip()
        if not word:
            continue

        line_no = (data["block_num"][i], data["par_num"][i], data["line_num"][i])
        x, y = data["left"][i], data["top"][i]

        if line_no != current_line:
            # flush previous line
            if line_text:
                html_parts.append(
                    f"<div style='position:absolute;left:{line_left}px;top:{line_top}px;"
                    "font-size:12px;white-space:nowrap;'>"
                    f"{' '.join(line_text)}</div>"
                )
            # start new line
            current_line = line_no
            line_text, line_left, line_top = [word], x, y
        else:
            line_text.append(word)

    # flush last line
    if line_text:
        html_parts.append(
            f"<div style='position:absolute;left:{line_left}px;top:{line_top}px;"
            "font-size:12px;white-space:nowrap;'>"
            f"{' '.join(line_text)}</div>"
        )

    html_parts.append("</div>")  # end page

# Final HTML skeleton
html_output = f"""
<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>Bank Statement</title>
<style>
  body {{ background:#f9f9f9; font-family:Arial, sans-serif; }}
  .page {{ box-shadow:0 0 8px rgba(0,0,0,0.2); }}
</style>
</head>
<body>
{''.join(html_parts)}
</body>
</html>
"""

with open(output_html, "w", encoding="utf-8") as f:
    f.write(html_output)

print(f"✅ Full-page HTML saved to {output_html}")


✅ Full-page HTML saved to bank_statement_full.html
